In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import keras_metrics

print(tf.VERSION)
print(tf.keras.__version__)

1.12.0
2.1.6-tf


Using TensorFlow backend.


In [2]:
def as_keras_metric(method):
    import functools
    from tensorflow.keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

In [3]:
precision = as_keras_metric(tf.metrics.precision)
recall = as_keras_metric(tf.metrics.recall)

In [4]:
model = tf.keras.Sequential()

model.add(layers.Dense(64, activation = 'relu',bias_initializer=tf.keras.initializers.constant(2.0)))

model.add(layers.Dense(32, activation = 'relu',bias_initializer=tf.keras.initializers.constant(2.0),))

model.add(layers.Dense(5, activation = 'softmax'))

In [ ]:
#precision  = keras_metrics.precision()
#recall = keras_metrics.recall()

In [5]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy',precision, recall])

In [6]:
import pandas as pd
from  sklearn.model_selection import train_test_split

dataX = pd.read_csv('~/Datasets/kdddd/X_1.csv')
datay = pd.read_csv('~/Datasets/kdddd/y_1.csv')

data_X = dataX.astype('float32')
data_y = datay.astype('float32')

X_train, X_val, y_train, y_val = train_test_split(data_X.values,data_y.values,test_size = 0.2, random_state=42)

In [7]:
model.fit(X_train, y_train, epochs=10, batch_size=100,validation_data=(X_val, y_val))

Train on 395216 samples, validate on 98805 samples
Epoch 1/10
395216/395216 [==============================] - 6s 15us/step - loss: 0.1795 - acc: 0.9877 - precision: 0.4804 - recall: 0.9918 - val_loss: 0.1339 - val_acc: 0.9909 - val_precision: 0.4878 - val_recall: 0.9929
Epoch 2/10
395216/395216 [==============================] - 6s 14us/step - loss: 0.1328 - acc: 0.9912 - precision: 0.4854 - recall: 0.9931 - val_loss: 0.1276 - val_acc: 0.9916 - val_precision: 0.4843 - val_recall: 0.9932
Epoch 3/10
395216/395216 [==============================] - 6s 14us/step - loss: 0.1302 - acc: 0.9914 - precision: 0.4831 - recall: 0.9932 - val_loss: 0.1257 - val_acc: 0.9919 - val_precision: 0.4819 - val_recall: 0.9933
Epoch 4/10
395216/395216 [==============================] - 6s 14us/step - loss: 0.1262 - acc: 0.9919 - precision: 0.4809 - recall: 0.9933 - val_loss: 0.1298 - val_acc: 0.9913 - val_precision: 0.4804 - val_recall: 0.9932
Epoch 5/10
395216/395216 [==============================] - 6s 14

In [ ]:
scores =  model.evaluate(X_val, y_val) 
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
@as_keras_metric
def auc_pr(y_true, y_pred, curve='PR'):
    return tf.metrics.auc(y_true, y_pred, curve)